In [4]:
import torch
import torch.nn as nn
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()

import pandas as pd
pd.set_option('display.max_columns', None)

In [45]:
def toTorchSparse(x):
    x = x.tocoo()
    x = torch.sparse.LongTensor(torch.LongTensor([x.row.tolist(), x.col.tolist()]),
                              torch.LongTensor(x.data.astype(np.int32)))
    return x

In [52]:
raw_train = pd.read_csv('data/train.csv')
raw_val = pd.read_csv('data/dev.csv')
raw_test = pd.read_csv('data/test.csv')

n_train = raw_train.shape[0]
n_val = raw_val.shape[0]
n_test = raw_test.shape[0]

raw = pd.concat([raw_train, raw_test, raw_test])
x_raw = vectorizer.fit_transform(raw['text'])
x_train = toTorchSparse(x_raw[:n_train])
x_val = torch.from_numpy(x_raw[n_train : n_train + n_val].toarray())
x_test = torch.from_numpy(x_raw[n_train + n_val:].toarray())

y_raw = raw['Category']
y_train = torch.from_numpy(y_raw[:n_train].to_numpy()).view(-1, 1)
y_val = torch.from_numpy(y_raw[n_train : n_train + n_val].to_numpy()).view(-1, 1)
y_test = torch.from_numpy(y_raw[n_train + n_val:].to_numpy()).view(-1, 1)

In [59]:
x_raw[n_train : n_train + n_val].toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [55]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = x_train.shape[1]
hidden_size = 1000 
num_classes = 1
num_epochs = 5
batch_size = 1000
learning_rate = 0.001

train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True) 
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False) 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False) 

In [8]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        #self.relu = nn.ReLU()
        #self.l2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()  
    
    def forward(self, x):
        out = self.l1(x)
        #out = self.relu(out)
        #out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.sigmoid(out)
        return out

In [9]:

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (bow, labels) in enumerate(train_loader): 
        bow = bow.type(torch.FloatTensor) .to_dense().to(device)
        labels = labels.type(torch.FloatTensor) .to(device)
        
        
        # Forward pass
        outputs = model(bow)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [10/80], Loss: 0.5024
Epoch [1/5], Step [20/80], Loss: 0.4057
Epoch [1/5], Step [30/80], Loss: 0.4082
Epoch [1/5], Step [40/80], Loss: 0.3563
Epoch [1/5], Step [50/80], Loss: 0.3938
Epoch [1/5], Step [60/80], Loss: 0.3160
Epoch [1/5], Step [70/80], Loss: 0.3364
Epoch [1/5], Step [80/80], Loss: 0.3227
Epoch [2/5], Step [10/80], Loss: 0.2680
Epoch [2/5], Step [20/80], Loss: 0.2352
Epoch [2/5], Step [30/80], Loss: 0.2738
Epoch [2/5], Step [40/80], Loss: 0.2343
Epoch [2/5], Step [50/80], Loss: 0.2134
Epoch [2/5], Step [60/80], Loss: 0.2222
Epoch [2/5], Step [70/80], Loss: 0.2315
Epoch [2/5], Step [80/80], Loss: 0.2262
Epoch [3/5], Step [10/80], Loss: 0.1879
Epoch [3/5], Step [20/80], Loss: 0.1992
Epoch [3/5], Step [30/80], Loss: 0.1788
Epoch [3/5], Step [40/80], Loss: 0.2054
Epoch [3/5], Step [50/80], Loss: 0.2232
Epoch [3/5], Step [60/80], Loss: 0.1940
Epoch [3/5], Step [70/80], Loss: 0.1922
Epoch [3/5], Step [80/80], Loss: 0.1977
Epoch [4/5], Step [10/80], Loss: 0.1536


torch.Size([1000, 71379])
torch.Size([1000, 71379])
torch.Size([1000, 71379])
torch.Size([1000, 71379])
torch.Size([1000, 71379])
torch.Size([1000, 71379])
torch.Size([1000, 71379])
torch.Size([1000, 71379])
torch.Size([1000, 71379])
torch.Size([1000, 71379])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])
torch.Size([1000, 71386])


KeyboardInterrupt: 

In [61]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for bow, labels in train_loader:
        bow = bow.type(torch.FloatTensor).to(device)
        labels = labels.type(torch.FloatTensor).to(device)
        outputs = model(bow)
        
        # max returns (value ,index)
        predicted = outputs.data.round()

        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test samples: {acc} %')

KeyboardInterrupt: 